# Subaru Data Reduction Notebook

## Set the Reduction Parameters

In the box below, you specify various parameters of the reduction process. These include the root directory of the reduction process. Included here is the directory that contains all of the fits files to be used in the reduction.

The process includes only fits files produced using a specified filter. This filter is specified by the variable `filter_name`.

### Bias Removal
If you have them, bias files can be included in the image reduction. By default, the reduction process estimates bias from the overscan regions within each image file. To override this behavior,  set the variable `combined_bias_dir` to a directory containing the combined bias files. There must be 10 files in that directory, one for each detector.
### Coordinate Maps
Coordinate maps enable the alignment of the individual images to the Gaia DR3 catalog. There is one coordinate map for each detector, ten coordinate maps in all. The repo supplies coordinate maps in the directory `<repo_dir>/SubaruCoordinateMaps`. You can supply your own coordinate maps by changing the `coord_map_dir` variable below. THe value of this variable must contain a path to a directory containing ten `.coo` files. See `help(iraf.geomap)` for information regarding the construction of .coo files.

### Flats and Darks
These are not yet implemented.

### The Detectors
The names of the 10 detectors are: `chihiro`, `clarisse`, `fio`, `kiki`, `nausicaa`, `ponyo`, `san`, `satsuki`, `sheeta`, and `sophie`.

In [1]:
# some preliminaries
import os, sys

repo_dir = '/home/kevin/repos/ReipurthBallyProject' #directory where the repo was cloned
sys.path.append(repo_dir)

#observation parameters

clean = False # if true, remove all directories and start afresh

filter_name = 'N-A-L656' #process all files for this filter
rootdir = '/home/kevin/Documents/N1333'
raw_fits_dir = 'raw_fits' # where all the raw fits files live

fits_out = 'N1333_ha.fits' #name of the resulting image which will go rootdir

# set this to a directory containing coord maps if using non-default maps
coord_maps_dir = os.path.join(repo_dir,'SubaruCoordinateMaps')
#coord_maps_dir = '<your coordinate map directory>'

# if set to None, bias will be computed from overscan regions of the images
# set to a directory containing 10 median-combined bias files if you have them
combined_bias_dir = None
#combined_bias_dir = '/home/kevin/Documents/M8data/M8/combined_bias' 
#combined_bias_dir = '<your dir of combined bias files>

remove_cosmic_rays = False #True invokes ccdproc.cosmicray_lacosmic (adds 10 minutes to processing)

# list of comments to be included in the output fits file header
# to document the reduction
comments = ['Created using fits header from orig file',
            'all other parameters default']

In [2]:
from pyraf import iraf

from astropy.io import fits
import numpy as np 
import pandas as pd
from ccdproc import ImageFileCollection
from src.SubaruUtils import  subaru_reduction, obs_dirs
import shutil


In [3]:
kw = ['FRAMEID', 'EXP-ID', 'DATA-TYP', 'EXPTIME', 'FILTER01', 'DETECTOR']
raw_fits = ImageFileCollection(os.path.join(rootdir, raw_fits_dir), keywords=kw)

#did we get valid filter?
filters = raw_fits.values('FILTER01', unique=True)
if filter_name not in filters:
    print(f'Invalid filter name: {filter_name}')
    print(f'valid filters names are: {filters}')
    raise ValueError()

## Create Coordinate Transformation Maps

In [4]:
image_dir = os.path.join(rootdir, filter_name)
#zap the image directory if needed
if clean:
    #blow it all away
    try:
        shutil.rmtree(image_dir)
    except:
        pass
if not os.path.exists(image_dir):
    os.mkdir(image_dir)

os.chdir(image_dir)
sred = subaru_reduction(filter_name, rootdir)

dirs = obs_dirs(rootdir, filter_name)


In [5]:

try:
    shutil.rmtree(dirs['coord_maps'])
except:
    pass

coord_maps = [p for p in os.listdir(coord_maps_dir) if p.endswith('.coo')]
os.mkdir(dirs['coord_maps'])
for p in coord_maps:
    src = os.path.join(coord_maps_dir, p)
    dst = os.path.join(dirs['coord_maps'], p)
    shutil.copy(src,dst)

detectors = ['chihiro', 'clarisse', 'fio', 'kiki', 'nausicaa', 'ponyo', 'san', 'satsuki', 'sheeta', 'sophie']

for d in detectors:
    res, res_df = sred.map_detector(d, degree=3)


## Remove Bias and Overscan Regions

This takes 2 minutes. The de-biased images are in the directory `<rootdir>/<filter_name>/no_bias`

In [6]:
from src.no_bias import remove_oscan
import warnings

try:
    shutil.rmtree(dirs['no_bias'])
except:
    pass
os.mkdir(dirs['no_bias'])

image_filter = {'DATA-TYP':'OBJECT', 'FILTER01': filter_name}
im_files = raw_fits.files_filtered(include_path=True, **image_filter)

for imf in im_files:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        #need the real header, apparently CCDData.read doesn't return WCS in header
        with fits.open(imf) as hdul:
            hdr = hdul[0].header.copy()
            data = hdul[0].data.astype(np.float32)

        detector = hdr['DETECTOR']
        print(f'file: {os.path.basename(imf)}, detector: {detector}')

        if combined_bias_dir is not None:
            bias_path = os.path.join(combined_bias_dir, detector+ '.fits')
            with fits.open(bias_path) as f:
                bias = f[0].data.copy()
        else:
            bias = None

        new_hdr, no_oscan = remove_oscan(hdr, data, bias)

        phdu = fits.PrimaryHDU(data = no_oscan, header=new_hdr)
        outfile = os.path.join(dirs['no_bias'], os.path.basename(imf))
        phdu.writeto(outfile, overwrite=True)

file: SUPA01425710.fits, detector: nausicaa
file: SUPA01425711.fits, detector: kiki
file: SUPA01425712.fits, detector: fio
file: SUPA01425713.fits, detector: sophie
file: SUPA01425714.fits, detector: sheeta
file: SUPA01425715.fits, detector: satsuki
file: SUPA01425716.fits, detector: chihiro
file: SUPA01425717.fits, detector: clarisse
file: SUPA01425718.fits, detector: ponyo
file: SUPA01425719.fits, detector: san
file: SUPA01425720.fits, detector: nausicaa
file: SUPA01425721.fits, detector: kiki
file: SUPA01425722.fits, detector: fio
file: SUPA01425723.fits, detector: sophie
file: SUPA01425724.fits, detector: sheeta
file: SUPA01425725.fits, detector: satsuki
file: SUPA01425726.fits, detector: chihiro
file: SUPA01425727.fits, detector: clarisse
file: SUPA01425728.fits, detector: ponyo
file: SUPA01425729.fits, detector: san
file: SUPA01425730.fits, detector: nausicaa
file: SUPA01425731.fits, detector: kiki
file: SUPA01425732.fits, detector: fio
file: SUPA01425733.fits, detector: sophie
f

## Register the Images

This takes about 3 minutes unless `remove_cosmic_rays = True` in which case it takes about 15 minutes. The results are in the directory `<rootdir>/<filter_name>/registered_image`

In [7]:
from ccdproc import ImageFileCollection
try:
    shutil.rmtree(dirs['registered_image'])
except:
    pass
os.mkdir(dirs['registered_image'])

imgs = ImageFileCollection(dirs['no_bias'])

for img in imgs.files:
    print(f'Tranforming: {img}')
    res=sred.transform_image(os.path.splitext(img)[0], remove_cosmic_rays = remove_cosmic_rays)

Tranforming: SUPA01425710.fits
Tranforming: SUPA01425711.fits
Tranforming: SUPA01425712.fits
Tranforming: SUPA01425713.fits
Tranforming: SUPA01425714.fits
Tranforming: SUPA01425715.fits
Tranforming: SUPA01425716.fits
Tranforming: SUPA01425717.fits
Tranforming: SUPA01425718.fits
Tranforming: SUPA01425719.fits
Tranforming: SUPA01425720.fits
Tranforming: SUPA01425721.fits
Tranforming: SUPA01425722.fits
Tranforming: SUPA01425723.fits
Tranforming: SUPA01425724.fits
Tranforming: SUPA01425725.fits
Tranforming: SUPA01425726.fits
Tranforming: SUPA01425727.fits
Tranforming: SUPA01425728.fits
Tranforming: SUPA01425729.fits
Tranforming: SUPA01425730.fits
Tranforming: SUPA01425731.fits
Tranforming: SUPA01425732.fits
Tranforming: SUPA01425733.fits
Tranforming: SUPA01425734.fits
Tranforming: SUPA01425735.fits
Tranforming: SUPA01425736.fits
Tranforming: SUPA01425737.fits
Tranforming: SUPA01425738.fits
Tranforming: SUPA01425739.fits
Tranforming: SUPA01425740.fits
Tranforming: SUPA01425741.fits
Tranform

## Project Individual Frames

This sub-process takes about 12 minutes.

In [8]:
from MontagePy.main import mImgtbl

imgdir = dirs['registered_image']
raw_image_tbl = os.path.join(image_dir, 'raw_image.tbl')

rtn = mImgtbl(imgdir, raw_image_tbl)
rtn

{'status': '0', 'count': 50, 'badfits': 0, 'badwcs': 0}

In [9]:
from MontagePy.main import mMakeHdr, mProjExec, mAdd
hdrfile = os.path.join(image_dir, 'mosaic.hdr')
rtn = mMakeHdr(raw_image_tbl, hdrfile )
rtn

{'status': '0',
 'count': 50,
 'ncube': 0,
 'naxis1': 11271,
 'naxis2': 9226,
 'clon': 52.27228381797356,
 'clat': 31.267232105974955,
 'lonsize': 0.63241581,
 'latsize': 0.5176708600000001,
 'posang': 359.9854226574486,
 'lon1': 52.64112899939523,
 'lat1': 31.00779378253605,
 'lon2': 51.90328434964749,
 'lat2': 31.00795423993255,
 'lon3': 51.90140888870261,
 'lat3': 31.525611213039923,
 'lon4': 52.643312617879936,
 'lat4': 31.525449872936914}

In [10]:
try:
    shutil.rmtree(dirs['projected_image'])
except:
    pass
os.mkdir(dirs['projected_image'])
projdir = dirs['projected_image']
rtn = mProjExec(imgdir, raw_image_tbl, hdrfile, projdir=projdir, quickMode=True)
rtn

{'status': '0', 'count': 50, 'failed': 0, 'nooverlap': 0}

## Create the Final Mosaic

This takes about one minute.

In [11]:
projdir = dirs['projected_image']
pimage_tbl = os.path.join(image_dir, 'pimages.tbl')

rtn = mImgtbl(projdir, pimage_tbl )
print(f'mImgtbl returned: {rtn}')

#coadd into a temp file
tmp_out = os.path.join(image_dir, 'tmp_mosaic.fits')

rtn = mAdd(projdir, pimage_tbl,  hdrfile, tmp_out, coadd=1)
print(f'mAdd returned: {rtn}')

# convert to single precision
mosaic_fits = os.path.join(rootdir,  fits_out)
with fits.open(tmp_out) as f:
    img_hdr=f[0].header.copy()
    img_data = f[0].data.astype(np.float32)

# tack on the comments to the header
img_hdr['COMMENT'] = '----------- Observation Comments -----------------'
for c in comments:
    img_hdr['COMMENT'] = c

phdu = fits.PrimaryHDU(data = img_data, header = img_hdr)
phdu.writeto(mosaic_fits, overwrite=True)

try:
    os.remove(tmp_out)
    os.remove(os.path.join(rootdir, filter_name, 'tmp_mosaic_area.fits'))
except:
    pass

mImgtbl returned: {'status': '0', 'count': 50, 'badfits': 0, 'badwcs': 0}
mAdd returned: {'status': '0', 'time': 44.0}
